<a href="https://colab.research.google.com/github/slunara/corporateProject/blob/main/simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Revenue Forecast

In [142]:
import numpy as np
import pandas as pd


def compute_customer_forecast(n_months, traffic, prob_prospect_generation,
                              prob_prospect_conversion,
                              prob_direct_customer_conversion, retention_prob, prob_existing_clients_conversion,
                              existing_customers):
    prospects = np.zeros(n_months)
    new_customers_from_prospects = np.zeros(n_months)
    new_customers_direct = np.zeros(n_months)
    total_existing_customers = np.zeros(n_months)

    prospects = traffic * prob_prospect_generation
    rows, cols = np.meshgrid(np.arange(n_months), np.arange(n_months), indexing="ij")
    M_conversion_matrix_ops = np.where(cols <= rows, prob_prospect_conversion[rows-cols], 0)
    new_customers_from_prospects = M_conversion_matrix_ops @ prospects
    new_customers_direct = traffic * prob_direct_customer_conversion
    M_retention_matrix_ops = np.where(cols <= rows, retention_prob[rows-cols], 0)
    retained_customers_from_prospects = M_retention_matrix_ops @ new_customers_from_prospects
    retained_customers_direct = M_retention_matrix_ops @ new_customers_direct
    existing_customers_vector = np.full(n_months, existing_customers * prob_existing_clients_conversion)
    total_existing_customers = existing_customers_vector + retained_customers_from_prospects + retained_customers_direct
    forecast_df = pd.DataFrame({
    "prospects": prospects,
    "new_customers_from_prospects": new_customers_from_prospects,  # Sum across rows
    "new_customers_direct": new_customers_direct,
    "total_existing_customers": total_existing_customers
    }, index=[f"Month {i+1}" for i in range(n_months)])

    return forecast_df

def compute_revenue_forecast(local_customer_forecast_df, tourist_customer_forecast_df, avg_ticket_df):

    total_revenue_df = pd.DataFrame(index=local_customer_forecast_df.index)

    total_revenue_df['revenue_local_new_from_prospects'] = local_customer_forecast_df['new_customers_from_prospects']*avg_ticket_df['local_avg_ticket_new_from_prospects'].iloc[0]
    total_revenue_df['revenue_local_new_from_direct'] = local_customer_forecast_df['new_customers_direct']*avg_ticket_df['local_avg_ticket_new_direct'].iloc[0]
    total_revenue_df['revenue_local_existing'] = local_customer_forecast_df['total_existing_customers']*avg_ticket_df['local_avg_ticket_existing'].iloc[0]
    total_revenue_df['revenue_local_total'] = total_revenue_df['revenue_local_new_from_prospects']+total_revenue_df['revenue_local_new_from_direct']+total_revenue_df['revenue_local_existing']

    total_revenue_df['revenue_tourist_new_from_prospects'] = tourist_customer_forecast_df['new_customers_from_prospects']*avg_ticket_df['tourist_avg_ticket_new_from_prospects'].iloc[0]
    total_revenue_df['revenue_tourist_new_from_direct'] = tourist_customer_forecast_df['new_customers_direct']*avg_ticket_df['tourist_avg_ticket_new_direct'].iloc[0]
    total_revenue_df['revenue_tourist_existing'] = tourist_customer_forecast_df['total_existing_customers']*avg_ticket_df['tourist_avg_ticket_existing'].iloc[0]
    total_revenue_df['revenue_tourist_total'] = total_revenue_df['revenue_tourist_new_from_prospects']+total_revenue_df['revenue_tourist_new_from_direct']+total_revenue_df['revenue_tourist_existing']

    total_revenue_df['revenue_total'] = total_revenue_df['revenue_local_total']+total_revenue_df['revenue_tourist_total']

    return total_revenue_df

In [143]:
# ------------------------------ #
# Example usage for Local & Tourist Customers
# ------------------------------ #

n_months = 6
traffic = np.array([1000, 1200, 1100, 1300, 1250, 1400])  # traffic
prob_prospect_generation = 0.30
prob_prospect_conversion = np.array([0.3, 0.2, 0.1, 0, 0, 0])  # 30% of traffic converts into prospects

tourist_prob_prospect_generation = 0

local_prob_direct_customer_conversion = 0.1  # 10% of traffic converts directly into customers
tourist_prob_direct_customer_conversion = 0.05

local_retention_prob = np.array([1.0, 0.7, 0.5, 0.3, 0.2, 0.1])  # Retention probability for new customers
tourist_retention_prob = np.array([0, 0.7, 0.5, 0.3, 0.2, 0.1])  # Retention probability for new customers

local_prob_existing_clients_conversion = 0.03
tourist_prob_existing_clients_conversion = 0.01

existing_local_customers = 5000  # Existing local customers before the forecast
existing_tourist_customers = 1000  # Existing tourist customers before the forecast

local_avg_ticket_new_from_prospects = 50
local_avg_ticket_new_direct = 60
local_avg_ticket_existing = 40

tourist_avg_ticket_new_from_prospects = 0
tourist_avg_ticket_new_direct = 80
tourist_avg_ticket_existing = 45

avg_ticket_df = pd.DataFrame({
    "local_avg_ticket_new_from_prospects": [local_avg_ticket_new_from_prospects],
    "local_avg_ticket_new_direct": [local_avg_ticket_new_direct],
    "local_avg_ticket_existing": [local_avg_ticket_existing],
    "tourist_avg_ticket_new_from_prospects": [tourist_avg_ticket_new_from_prospects],
    "tourist_avg_ticket_new_direct": [tourist_avg_ticket_new_direct],
    "tourist_avg_ticket_existing": [tourist_avg_ticket_existing]
})


# Compute forecasts for Local Customers
local_customer_forecast_df = compute_customer_forecast(
    n_months, traffic, prob_prospect_generation,prob_prospect_conversion,
    local_prob_direct_customer_conversion, local_retention_prob, local_prob_existing_clients_conversion,
    existing_local_customers
)

# Compute forecasts for Tourist Customers
tourist_customer_forecast_df = compute_customer_forecast(
    n_months, traffic, tourist_prob_prospect_generation,  prob_prospect_conversion,
    tourist_prob_direct_customer_conversion, tourist_retention_prob, tourist_prob_existing_clients_conversion,
    existing_tourist_customers
)

total_revenue=compute_revenue_forecast(local_customer_forecast_df,tourist_customer_forecast_df,avg_ticket_df)

In [144]:
total_revenue

,revenue_local_new_from_prospects,revenue_local_new_from_direct,revenue_local_existing,revenue_local_total,revenue_tourist_new_from_prospects,revenue_tourist_new_from_direct,revenue_tourist_existing,revenue_tourist_total,revenue_total
Month 1,4500.0,6000.0,13600.0,24100.0,0.0,4000.0,450.00,4450.00,28550.00
Month 2,8400.0,7200.0,22840.0,38440.0,0.0,4800.0,2025.00,6825.00,45265.00
Month 3,10050.0,6600.0,30304.0,46954.0,0.0,4400.0,3465.00,7865.00,54819.00
Month 4,10950.0,7800.0,36708.0,55458.0,0.0,5200.0,4207.50,9407.50,64865.50
Month 5,11175.0,7500.0,40908.0,59583.0,0.0,5000.0,4995.00,9995.00,69578.00
Month 6,12000.0,8400.0,44734.0,65134.0,0.0,5600.0,5388.75,10988.75,76122.75


In [145]:
local_customer_forecast_df.sum()

,0
prospects,2175.00
new_customers_from_prospects,1141.50
new_customers_direct,725.00
total_existing_customers,4727.35


In [146]:
tourist_customer_forecast_df.sum()

,0
prospects,0.00
new_customers_from_prospects,0.00
new_customers_direct,362.50
total_existing_customers,456.25


# Sensitivity Analysis

Identify the most influential variable for revenue

In [147]:
variables = {
    "local_prob_prospect_generation": prob_prospect_generation,
    "local_prob_prospect_conversion": prob_prospect_conversion,
    "local_prob_direct_customer_conversion": local_prob_direct_customer_conversion,
    "local_prob_existing_clients_conversion": local_prob_existing_clients_conversion,
    "local_retention_prob":local_retention_prob,

    "tourist_prob_direct_customer_conversion":tourist_prob_direct_customer_conversion,
    "tourist_prob_existing_clients_conversion": tourist_prob_existing_clients_conversion,
    "tourist_retention_prob":tourist_retention_prob
}

In [148]:
# Define baseline total revenue
baseline_total_revenue = total_revenue['revenue_total'].sum()
# Store sensitivity results
sensitivity_results = {}

# Loop through each variable to test its sensitivity
for var in variables.keys():
    # Create a copy of the original variables (fresh start each time)

    modified_variables = {k: v.copy() if isinstance(v, np.ndarray) else v for k, v in variables.items()}


    # Apply a 1% increase ONLY to the current variable
    modified_variables[var] = variables[var] * 1.01  # Increase by 1%


    # Compute forecasts using modified variables
    local_customer_forecast_df = compute_customer_forecast(
        n_months, traffic,
        modified_variables["local_prob_prospect_generation"],
        modified_variables["local_prob_prospect_conversion"],
        modified_variables["local_prob_direct_customer_conversion"],
        modified_variables["local_retention_prob"],
        modified_variables["local_prob_existing_clients_conversion"],
        existing_local_customers
    )

    tourist_customer_forecast_df = compute_customer_forecast(
        n_months, traffic,
        prob_prospect_generation,
        prob_prospect_conversion,
        modified_variables["tourist_prob_direct_customer_conversion"],
        modified_variables["tourist_retention_prob"],
        modified_variables["tourist_prob_existing_clients_conversion"],
        existing_tourist_customers
    )

    # Compute new total revenue
    new_total_revenue_df = compute_revenue_forecast(local_customer_forecast_df, tourist_customer_forecast_df, avg_ticket_df)

    # Extract sum of revenue
    new_total_revenue = new_total_revenue_df['revenue_total'].sum()


    # Calculate percentage impact
    sensitivity_results[var] = ((new_total_revenue - baseline_total_revenue) / baseline_total_revenue) * 100

# Convert results to DataFrame and display
sensitivity_df = pd.DataFrame.from_dict(sensitivity_results, orient='index', columns=["% Impact on Total Revenue"])
sensitivity_df = sensitivity_df.sort_values(by="% Impact on Total Revenue", ascending=False)

In [149]:
sensitivity_df

,% Impact on Total Revenue
local_retention_prob,15.951253
local_prob_prospect_generation,15.940566
local_prob_prospect_conversion,15.940566
local_prob_direct_customer_conversion,15.807108
tourist_retention_prob,15.707483
tourist_prob_direct_customer_conversion,15.637979
local_prob_existing_clients_conversion,15.606047
tourist_prob_existing_clients_conversion,15.507875


# Revenue GAP Forecast vs Budget

In [150]:
sales_budget=400000

In [151]:
def compute_new_forecast(sales_budget,sensitivity_df,baseline_total_revenue):

    # Calcular el porcentaje de crecimiento necesario
    growth_required = ((sales_budget - baseline_total_revenue) / baseline_total_revenue) * 100

    print(f"Alert: It is required a growth in sales {growth_required:.2f}% to reach sales budget.")
    incrementPerAction= (0.01*growth_required/sensitivity_df)*100
    incrementPerAction = incrementPerAction.rename(columns={"% Impact on Total Revenue": "Required Percentage Change"})

    return incrementPerAction


In [152]:
incrementPerAction=compute_new_forecast(sales_budget,sensitivity_df,baseline_total_revenue)

Alert: It is required a growth in sales 17.92% to reach sales budget.


In [153]:
incrementPerAction

,Required Percentage Change
local_retention_prob,1.123701
local_prob_prospect_generation,1.124455
local_prob_prospect_conversion,1.124455
local_prob_direct_customer_conversion,1.133948
tourist_retention_prob,1.141140
tourist_prob_direct_customer_conversion,1.146212
local_prob_existing_clients_conversion,1.148557
tourist_prob_existing_clients_conversion,1.155828


# Compute the new forecast

In [154]:
variable_To_Change = incrementPerAction.index[0]  # Get the first row's index (variable name)
PO_change = incrementPerAction.iloc[0, 0]  # Get the first row's value

print(f"Variable: {variable_To_Change}, Value: {PO_change}")

Variable: local_retention_prob, Value: 1.1237011541456323


In [155]:
def compute_new_forecast(variable_To_Change, PO_change):
    """
    Computes the total revenue after modifying ONLY the given variable by PO_change.

    Parameters:
    - variable_To_Change (str): The name of the variable to be modified.
    - PO_change (float): The percentage change to be applied.

    Returns:
    - new_total_revenue (DataFrame): Updated revenue forecast.
    """
    # Create a copy of the original variable
    updated_variable_value = globals()[variable_To_Change] * (1 + PO_change)  # Apply % change

    # Debugging print statement
    print(f"🔍 Changing Variable: {variable_To_Change} | Original: {globals()[variable_To_Change]} -> New: {updated_variable_value}")

    # Recalculate forecasts using updated variable
    local_customer_forecast_df = compute_customer_forecast(
        n_months, traffic,
        prob_prospect_generation if variable_To_Change != "prob_prospect_generation" else updated_variable_value,
        prob_prospect_conversion if variable_To_Change != "prob_prospect_conversion" else updated_variable_value,
        local_prob_direct_customer_conversion if variable_To_Change != "local_prob_direct_customer_conversion" else updated_variable_value,
        local_retention_prob if variable_To_Change != "local_retention_prob" else updated_variable_value,
        local_prob_existing_clients_conversion if variable_To_Change != "local_prob_existing_clients_conversion" else updated_variable_value,
        existing_local_customers if variable_To_Change != "existing_local_customers" else updated_variable_value
    )

    tourist_customer_forecast_df = compute_customer_forecast(
        n_months, traffic,
        prob_prospect_generation if variable_To_Change != "prob_prospect_generation" else updated_variable_value,
        prob_prospect_conversion if variable_To_Change != "prob_prospect_conversion" else updated_variable_value,
        tourist_prob_direct_customer_conversion if variable_To_Change != "tourist_prob_direct_customer_conversion" else updated_variable_value,
        tourist_retention_prob if variable_To_Change != "tourist_retention_prob" else updated_variable_value,
        tourist_prob_existing_clients_conversion if variable_To_Change != "tourist_prob_existing_clients_conversion" else updated_variable_value,
        existing_tourist_customers if variable_To_Change != "existing_tourist_customers" else updated_variable_value
    )

    # Compute new total revenue
    new_total_revenue_df = compute_revenue_forecast(local_customer_forecast_df, tourist_customer_forecast_df, avg_ticket_df)

    return new_total_revenue_df,local_customer_forecast_df,tourist_customer_forecast_df


In [156]:
new_total_revenue_df,local_customer_forecast_df,tourist_customer_forecast_df=compute_new_forecast(variable_To_Change, PO_change)
new_total_revenue = new_total_revenue_df['revenue_total'].sum()
new_total_revenue

🔍 Changing Variable: local_retention_prob | Original: [1.  0.7 0.5 0.3 0.2 0.1] -> New: [2.12370115 1.48659081 1.06185058 0.63711035 0.42474023 0.21237012]


np.float64(563807.9044927714)

In [157]:
new_total_revenue_df.sum()

,0
revenue_local_new_from_prospects,57075.000000
revenue_local_new_from_direct,43500.000000
revenue_local_existing,361125.904493
revenue_local_total,461700.904493
revenue_tourist_new_from_prospects,0.000000
revenue_tourist_new_from_direct,29000.000000
revenue_tourist_existing,73107.000000
revenue_tourist_total,102107.000000
revenue_total,563807.904493


In [158]:
local_customer_forecast_df.sum()

,0
prospects,2175.000000
new_customers_from_prospects,1141.500000
new_customers_direct,725.000000
total_existing_customers,9028.147612


In [159]:
tourist_customer_forecast_df.sum()

,0
prospects,2175.0
new_customers_from_prospects,1141.5
new_customers_direct,362.5
total_existing_customers,1624.6


# KPI re-calculation

In [160]:
new_total_revenue_df

,revenue_local_new_from_prospects,revenue_local_new_from_direct,revenue_local_existing,revenue_local_total,revenue_tourist_new_from_prospects,revenue_tourist_new_from_direct,revenue_tourist_existing,revenue_tourist_total,revenue_total
Month 1,4500.0,6000.0,22140.128772,32640.128772,0.0,4000.0,450.0,4450.0,37090.128772
Month 2,8400.0,7200.0,41763.127436,57363.127436,0.0,4800.0,4860.0,9660.0,67023.127436
Month 3,10050.0,6600.0,57614.432850,74264.432850,0.0,4400.0,10782.0,15182.0,89446.432850
Month 4,10950.0,7800.0,71214.615042,89964.615042,0.0,5200.0,15534.0,20734.0,110698.615042
Month 5,11175.0,7500.0,80134.159889,98809.159889,0.0,5000.0,19494.0,24494.0,123303.159889
Month 6,12000.0,8400.0,88259.440505,108659.440505,0.0,5600.0,21987.0,27587.0,136246.440505


In [161]:
new_total_revenue_sum=new_total_revenue_df.sum()

In [162]:
new_total_revenue_sum

,0
revenue_local_new_from_prospects,57075.000000
revenue_local_new_from_direct,43500.000000
revenue_local_existing,361125.904493
revenue_local_total,461700.904493
revenue_tourist_new_from_prospects,0.000000
revenue_tourist_new_from_direct,29000.000000
revenue_tourist_existing,73107.000000
revenue_tourist_total,102107.000000
revenue_total,563807.904493


## not done yet

In [163]:
def compute_kpi(local_new_direct,traffic,local_new_from_prospects,local_existing,existing_local_customers,tourist_new_direct,tourist_existing,existing_tourist_customers):
    local_new_closing_ratio=local_new_direct.sum().values[0]/traffic.sum()
    prospect_closing_ratio=local_new_from_prospects.sum().values[0]/traffic.sum()
    local_come_back=local_existing.sum().values[0]/(existing_local_customers+local_existing.sum().values[0])
    tourist_new_closing_ratio=tourist_new_direct.sum().values[0]/traffic.sum()
    tourist_come_back=tourist_existing.sum()/(existing_tourist_customers+tourist_existing.sum().values[0])
    kpi_df=pd.DataFrame([local_new_closing_ratio,prospect_closing_ratio,local_come_back,tourist_new_closing_ratio, tourist_come_back])
    return kpi_df


In [164]:
def compute_kpi(traffic,existing_local_customers,existing_tourist_customers,total_revenue_df):
    local_new_closing_ratio=total_revenue_df['revenue_local_new_from_direct'].sum().values[0]/traffic.sum()
    prospect_closing_ratio=total_revenue_df['revenue_local_new_from_prospects'].values[0]/traffic.sum()
    local_come_back=total_revenue_df['revenue_local_existing'].values[0]/(existing_local_customers+total_revenue_df['revenue_local_new_from_direct'].values[0]+)
    tourist_new_closing_ratio=tourist_new_direct.sum().values[0]/traffic.sum()
    tourist_come_back=tourist_existing.sum()/(existing_tourist_customers+tourist_existing.sum().values[0])

    kpi_df=pd.DataFrame([local_new_closing_ratio,prospect_closing_ratio,local_come_back,tourist_new_closing_ratio, tourist_come_back])
    return kpi_df

kpi_df=compute_kpi(traffic,existing_local_customers,existing_tourist_customers,total_revenue_df)

SyntaxError: invalid syntax (<ipython-input-164-f915514df4a4>, line 4)